In the first part of this assignment, we will implement Gaussian Mixture Model and Expectation Maximization.

### **Preparation**
The following code is the preparation for importing packages. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal as mvn
from sklearn.cluster import KMeans

### **Gaussian mixture model and Expectation maximization Implementation**

In this assignment, you should use only NumPy to build the Gaussian mixture models.

**DO NOT** use other libraries to implement the Gaussian mixture models.

**DO NOT** modify other parts of the skeleton code.

Follow the comments. They'll give you instructions on what to code.

### Step 1. Implement EM for GMM
The followings are the skeleton codes. Implement the functions inside the GMM class.

In [ ]:
class GMM():
    def __init__(self, X_data, cluster_idx):
        # Initializet the model parameters as the instance variable.
        self.n, self.p = X_data.shape
        self.num1 = cluster_idx[0]
        self.num2 = cluster_idx[1]
        self.mean = np.random.random((self.p, self.p))
        self.covariance = np.array([np.eye(2)] * 2)
        self.mixing_coefficient = np.random.random(len(cluster_idx))
        self.mixing_coefficient /= self.mixing_coefficient.sum()
        self.E = np.zeros((len(cluster_idx), self.n))
    
    # Function for computing log likelihood.
    def log_likelihood(self, X_data):
        k = len(self.mixing_coefficient)
        result = 0
        for i in range(self.n):
            # Initialize the probability as zero.
            prob = 0
            for j in range(k):
                # Multiply the probability density function with the mixing coefficient.
                prob += self.mixing_coefficient[j] * mvn(self.mean[j], self.covariance[j]).pdf(X_data[i])
            # Add the log-likelihood values to the output.
            result += np.log(prob)
        return result

    # Function for expectation step.
    def E_step(self, X_data):
        # Compute the posterior probability.
        k = len(self.mixing_coefficient)
        for i in range(len(self.mean)):
            for j in range(self.n):
                # In order to adgust the parameter,Compute the posterior probability over all possibilities.
                self.E[i,j] = self.mixing_coefficient[i] * mvn(self.mean[i], self.covariance[i]).pdf(X_data[j])
        # Normalize the posterior probability for each data point.
        self.E /= self.E.sum(0)

    # Function for maximization step.
    def M_step(self, X_data):
        k = len(self.mixing_coefficient)
        # Update the mixing coefficient vector using the result of the E step.
        for i in range(k):
            for j in range(self.n):
                self.mixing_coefficient[i] += self.E[i, j]
        # Normalize the mixing coefficient.
        self.mixing_coefficient /= self.n

        # Update the mean vector.
        for i in range(k):
            for j in range(self.n):
                # Multiply the each data point by the corresponding 𝛾𝑘(𝑛).
                self.mean[i] += self.E[i, j] * X_data[j]
            # Divide by the number of data points in the i-th distribution
            self.mean[i] /= self.E[i, :].sum()

        # Update the covariance matrix.
        for i in range(k):
            for j in range(self.n):
                # Subtract mean from the data point.
                tmp = np.reshape(X_data[j] - self.mean[i], (2,1))
                # Multiply the above matrix and multiply the gamma value.
                self.covariance[i] += self.E[i,j] * np.dot(tmp, tmp.T)
            # Divide by the number of data points in the i-th distribution.
            self.covariance[i] /= self.E[i,:].sum()

    
    def train(self, X_data, iterations=1000, threshold=0.001):
        # Initialize the log-likelihood value to start the training.
        self.P_new = self.log_likelihood(X_data)
        self.P_old = 2 * self.P_new
        iteration = 0
        while((abs((self.P_old - self.P_new)/self.P_new)*100) > threshold and (iteration <= iterations)):
            # Repeat the E and M step to update the parameters
            # until the difference in log-likelihood value between iterations, or total iterations end.
            self.E_step(X_data)
            self.M_step(X_data)
            self.P_old = self.P_new
            self.P_new = self.log_likelihood(X_data)
            iteration += 1
            

In [ ]:
def plot_for_EM(X_data, E, mean, mu_1, mu_2, num_1):
    # Generates the empty lists for each cluster.
    cl1, cl2 = list(), list()

    # We don't know the order of the points (due to the initialization),
    # so we make two variables (true1, true2) for count
    true1 = 0
    true2 = 0
    for i in range(len(X_data)):
        if E[i, 0] > E[i, 1]:
            cl1.append(X_data[i,:])
            if i < num_1:
                true1 += 1
            else:
                true2 += 1
        else:
            cl2.append(X_data[i,:])
            if i >= num_1:
                true1 += 1
            else:
                true2 += 1
                
    # Make lists into the NumPy array.
    cl1 = np.array(cl1)
    cl2 = np.array(cl2)
    index = 0 if true1 > true2 else 1
    color = ['b.', 'r.']
    
    # Compute the accuracy of the prediction for the EM algorithm and the center of each cluster.
    print(f'- The accuracy of the prediction for the EM algorithm : {float(max(true1, true2)) / len(X_data)}')
    print(f'- The estimated center of cluster 1 : ({mean[index,0]}, {mean[index,1]})')
    print(f'- The estimated center of cluster 2 : ({mean[1-index,0]}, {mean[1-index,1]})')
    
    fig = plt.figure(figsize=(9, 6), dpi=100)
    ax = fig.add_subplot(111)
    ax.scatter(cl1[:,0], cl1[:,1], label="Cluster 1")
    ax.scatter(cl2[:,0], cl2[:,1], label="Cluster 2")
    ax.plot(mean[0,0], mean[0,1], marker='*', color='black', linestyle='None',label="Estimated center")
    ax.plot(mean[1,0], mean[1,1], marker='*', color='black', linestyle='None')
    ax.plot(mu_1[0], mu_1[1], marker='+', color='black', linestyle='None', label="Ground truth center")
    ax.plot(mu_2[0], mu_2[1], marker='+', color='black', linestyle='None')
    plt.legend()
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title("The scatter plot of the prediction for EM algorithm", fontsize=17)
    plt.show()

### Step 2. Load the sample dataset
Load the sample dataset and visualize it using the scatter plot.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# The Dataset0(X_data).npy is generated by the multivariate gaussian distribution.
# To compute the accuracy and visualize the results, the graound truth information is as follows: 
mu_1 = [0, 4] # The mean vector of each group.
mu_2 = [-2, 0] # The mean vector of each group.
cluster_idx = [600, 400] # The number of samples of each group.
X_data = np.load("./drive/MyDrive/Dataset0(X_data).npy")

fig = plt.figure(figsize=(9, 6), dpi=100)
ax = fig.add_subplot(111)
ax.scatter(X_data[:,0], X_data[:,1], label="Sample dataset")
plt.legend()
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title("The scatter plot of the sample dataset", fontsize=17)
plt.show()

### Step 3. Run the EM algorithm on the sample dataset and visualize the prediction results
Initialize the GMM model with the sample data and the number of samples or each cluster. Set the total iterations and threshold to complete learning, and then run the EM algorithm

In [ ]:
model = GMM(X_data, cluster_idx = cluster_idx)
model.train(X_data, iterations = 1000, threshold=0.001)

In [ ]:
plot_for_EM(X_data, model.E.T, model.mean, mu_1, mu_2, model.num1)

### Step 4.  Visualize the prediction result of the K-Means algorithm for the sample dataset
Check the difference in the predicted center coordinates between the EM algorithm and the K-Means algorithm.

In [ ]:
# Run KMean Clustering
kmeans = KMeans(n_clusters = 2)
kmeans.fit(X_data)
labels = kmeans.predict(X_data)
kmean_center = kmeans.cluster_centers_

x = X_data[:, 0]
y = X_data[:, 1]
x_1, y_1 = x[labels==0], y[labels==0]
x_2, y_2 = x[labels==1], y[labels==1]
m_1 = np.array([np.average(x_1), np.average(y_1)])
m_2 = np.array([np.average(x_2), np.average(y_2)])

# We don't know the order of the points (due to the initialization), so we check the distance between center and the mean
true = 0.0
num = cluster_idx[0]
index = 0 if np.linalg.norm(m_1 - mu_1) < np.linalg.norm(m_1 - mu_2) else 1
true += len(labels[:num][labels[:num]==index])
true += len(labels[num:][labels[num:]==1 - index])
color = ['b.', 'r.']

# Compute the accuracy of the prediction for the N-Means algorithm and the center of each cluster.
print(f'- The accuracy of the prediction for the K-Means algorithm : {true / len(X_data)}')
print(f'- The estimated center of cluster 1 : ({kmean_center[index,0]}, {kmean_center[index,1]})')
print(f'- The estimated center of cluster 2 : ({kmean_center[1-index,0]}, {kmean_center[1-index,1]})')

fig = plt.figure(figsize=(9, 6), dpi=100)
ax = fig.add_subplot(111)
ax.scatter(x_1, y_1,label="Cluster 1")
ax.scatter(x_2, y_2, label="Cluster 2")
ax.plot(m_1[0], m_1[1], marker='*', color='black', linestyle='None',label="Estimated center")
ax.plot(m_2[0], m_2[1], marker='*', color='black', linestyle='None')
ax.plot(mu_1[0], mu_1[1], marker='+', color='black', linestyle='None', label="Ground truth center")
ax.plot(mu_2[0], mu_2[1], marker='+', color='black', linestyle='None')
plt.legend()
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title("The scatter plot of the prediction for K-Means algorithm", fontsize=17)
plt.show()